In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from src.config import CORRELATED_FEATURES_TO_REMOVE, LOG_TRANSFORM_FEATURES, NAME_COLS
from src.helpers import remove_correlations, log_trainsform, train_lgbm_default, drop_names, encode_categorical

In [2]:
from dotenv import load_dotenv

load_dotenv()

TRAIN_DATA_PATH = os.getenv("TRAIN_DATA_PATH")
TEST_DATA_PATH = os.getenv("TEST_DATA_PATH")


In [7]:
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)

In [4]:
df_train.shape

(6244474, 11)

## **Preprocessing**

- DO NOT remove outliers (give signal)

In [8]:
df_train = remove_correlations(df_train, CORRELATED_FEATURES_TO_REMOVE)
df_train = log_trainsform(df_train, LOG_TRANSFORM_FEATURES)
df_train = drop_names(df_train, NAME_COLS)
df_train = encode_categorical(df_train)[0]
df_train


c:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,step,amount,oldbalanceOrg,oldbalanceDest,urgency_level,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9.194276,12.044359,0.000000,0,0.0,0.0,0.0,1.0,0.0
1,1,7.531166,9.964112,0.000000,0,0.0,0.0,0.0,1.0,0.0
2,1,5.204007,5.204007,0.000000,1,0.0,0.0,0.0,0.0,1.0
3,1,5.204007,5.204007,9.960954,1,0.0,1.0,0.0,0.0,0.0
4,1,9.364703,10.634773,0.000000,0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6244469,595,12.165804,12.684927,16.288907,0,1.0,0.0,0.0,0.0,0.0
6244470,595,8.573832,13.151825,15.304099,0,1.0,0.0,0.0,0.0,0.0
6244471,595,11.645376,13.162046,14.038291,0,1.0,0.0,0.0,0.0,0.0
6244472,595,10.452698,13.360438,14.701242,0,1.0,0.0,0.0,0.0,0.0


**LightGBM model**

In [9]:
X_train = df_train.drop(columns=["urgency_level"])
y_train = df_train["urgency_level"]

model, metrics, _ = train_lgbm_default(X_train, y_train)
metrics

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030340 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1024
[LightGBM] [Info] Number of data points in the train set: 4995579, number of used features: 9
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[50]	valid_0's multi_logloss: 0.0777231
[100]	valid_0's multi_logloss: 0.0268364


KeyboardInterrupt: 